# ECE447 Control of a Gantry Crane

Eric Klavins

Copyright &copy; University of Washington, 2019

# Code

In [ ]:
import numpy as np
import scipy.integrate as spi
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sympy import *
from matplotlib import animation
from JSAnimation.IPython_display import display_animation
from control import * 

%matplotlib inline

# Comment out in Google colab 
init_printing(use_latex='mathjax')

# Uncomment in below Google colab to render sympy equations nicely
# def custom_latex_printer(exp,**options):
#     from google.colab.output._publish import javascript
#     url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
#     javascript(url=url)
#     return printing.latex(exp,**options)
# init_printing(use_latex="mathjax",latex_printer=custom_latex_printer)

def chop(A, eps = 1e-10):
	B = np.copy(A)
	B[np.abs(A) < eps] = 0
	return B

# Equations of Motion

Our goal in this section is to model a gantry crane.

<img width=35% src="https://static.turbosquid.com/Preview/2015/05/30__03_50_17/Rubber_Tyred_Gantry_Cranes_00.jpgd524e095-1af7-4810-83ba-2b8fbcecc9f3Original.jpg">

A model of this system consists of a mass $m_1$ that can move horizontally, with a load $m_2$ attached via a stiff beam of length L swings like a pendulum.

<img width=35% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/gantry-crane.png">

To find the equations of motion of this system, we will use the Lagrangian method.

**Kinetic Energy:** First we define the kinetic energy 

$$
K = \frac{1}{s}m_1 \dot x^2  + \frac{1}{s}m_2 \left ( \dot x_L^2 + \dot y_L^2 \right )
$$

where $x_L$ and $y_L$ are the position of the load, which are defined by

$$
x_L = x + L \sin \theta
$$

and

$$
y_L = -L\cos\theta)
$$

Thus, 

$$
K = \frac{1}{s}m_1 \dot x^2  + 
    \frac{1}{s}m_2 \left ( 
        \left ( \frac{d}{dt} (x + L \sin \theta) \right )^2 +
        \left ( \frac{d}{dt} (L-L\cos \theta) \right )^2 \right )
$$


**Potential Energy:**: The potential energy of the system is given by

$$
U = mgy_L = mgL(1-\cos\theta).
$$

**The Lagrangian:** The Lagrangian is defined to be

$$
L = K-U
$$

and Lagrange's equations in this case are

$$
\frac{d}{dt}\frac{\partial L}{\partial \dot x} - \frac{\partial L}{\partial x} = \mathrm{external\;forces\;on}\;x
$$

and

$$
\frac{d}{dt}\frac{\partial L}{\partial \dot \theta} - \frac{\partial L}{\partial \theta} = \mathrm{external\;forces\;on}\;\theta
$$

suppose that there is friction on the cart and pendulum, and that the control input is forces applied to the cart, we get

$$
\frac{d}{dt}\frac{\partial L}{\partial \dot x} - \frac{\partial L}{\partial x} = -k_1 \dot x + u
$$

and

$$
\frac{d}{dt}\frac{\partial L}{\partial \dot \theta} - \frac{\partial L}{\partial \theta} = - k_2 \dot \theta
$$

Computing these equations is easier with some computer help.

In [ ]:
var("m g L x theta t m1 m2 k1 k2 u v omega q w uu")

xdot = diff(x(t),t)
xddot = diff(xdot,t)
xdddot = diff(xddot,t)

thetadot = diff(theta(t),t)
thetaddot = diff(thetadot,t)
thetadddot = diff(thetaddot,t)

U = m2*g*(L-L*cos(theta(t)))

K = (1/2)*m1*diff(x(t),t)**2/2 +(1/2)*m2*(
      diff(x(t)+L*sin(theta(t)),t)**2 +
      diff((-L*cos(theta(t))),t)**2
    )

K = K.simplify()

Lagrangian = K-U
Lagrangian = Lagrangian.simplify()
Lagrangian

In [ ]:
# Apply Lagrange's equations, with external forces
eq1 = diff(diff(Lagrangian,xdot),t)-diff(Lagrangian,x(t)) + k1*xdot - u
eq2 = diff(diff(Lagrangian,thetadot),t)-diff(Lagrangian,theta(t)) + k2*thetadot
eq1,eq2

In [ ]:
# Now solve for x'' and theta''
sol = solve([eq1,eq2],[xddot,thetaddot])
sol

In [ ]:
# Here, we replace the variables names, which are all functions of time, with just symbols
# since subsequent calculations do not need the dependencies on time.
# Note: order matters. Must do xdot before x, etc.
sub = [            
    (xdot,v),    
    (x(t),x),
    (thetadot,w),
    (theta(t),q),
]
dv = sol[xddot].subs(sub)
dw = sol[thetaddot].subs(sub)
dv, dw

# Simulation Viewers

In [ ]:
def show_sim(tvals,X,title):
    
    fig,ax = plt.subplots(1,3,figsize=(15,5))

    plots = ax[0].plot(tvals,X[:,0],tvals,X[:,1]);
    ax[0].legend(plots, ["$x$", "$v$" ]);
    ax[0].set_xlabel("$t$")
    ax[0].set_title(title + "\nPosition / Velocity");

    plots = ax[1].plot(tvals,X[:,2],tvals,X[:,3]);
    ax[1].legend(plots, ["$x$", "$v$" ]);
    ax[1].set_xlabel("$t$")
    ax[1].set_title(title + "\nAngle / Angular Velocity");

    ax[2].plot(tvals,X[:,0]+10*np.sin(X[:,2]));
    ax[2].set_title(title + "\nLoad Position")
    ax[2].set_xlabel("$t$")
    ax[2].set_ylabel("$x(t)$");
    
    return fig, ax

def anim(X,title):
    
    # This function is only here so that all the variables below are local to it and
    # we don't overwrite them as defined above.
    
    x = X[:,0][0]
    q = X[:,2][0]
    l = 10

    xl = x+l*sin(q)
    yl = -l*cos(q)

    fig,ax = plt.subplots(1)

    beam, = ax.plot([x,xl],[0,yl])
    rail, = ax.plot([0,40],[0,0])
    cart = patches.Rectangle((x-1,-1), 2,2)
    load = patches.Circle((xl,yl), 1)

    ax.add_patch(cart)
    ax.add_patch(load)
    ax.set_xlim(0,40)
    ax.set_ylim(-15,5)
    ax.set_aspect(1)
    ax.set_title(title)

    num_frames = len(X)

    def animate(f):
        x = X[:,0][f]
        q = X[:,2][f] 
        xl = x+l*sin(q)
        yl = -l*cos(q)
        cart.set_xy((x-1,-1))
        load.set_center((xl,yl))
        beam.set_data([x,xl],[0,yl])
        return [cart,load,beam]

    anim = animation.FuncAnimation(fig, animate, frames=num_frames, interval=80)
    return display_animation(anim, default_mode='loop')

# Open Loop Simulation

In [ ]:
params = [
    (m1,1), (m2,0.1), (L,10), (k1,2), (k2,0.1), (g,9.81)
]

dynamics = lambdify([x,v,q,w,uu], [
    v, 
    dv.subs(params).subs(u,uu),
    w,
    dw.subs(params).subs(u,uu)
])

output = lambdify([x,v,q,w], (x + 10*sin(q)).subs(params))

def f_open_loop(X,t):
    x,v,q,w = X
    return dynamics(x,v,q,w,0)

tvals = np.linspace(0,100,200)
X = spi.odeint(f_open_loop,[15,5,0,0],tvals)

show_sim(tvals, X, "Open Loop Control");

In [ ]:
# Animation   
anim(X, "Open Loop Control")

# Linearization

In [ ]:
# Linearize the system
A = Matrix([
    [0,1,0,0],
    [diff(dv,x), diff(dv,v), diff(dv,q), diff(dv,w)],
    [0,0,0,1],
    [diff(dw,x), diff(dw,v), diff(dw,q), diff(dw,w)]
]).subs([(x,0),(v,0),(q,0),(w,0)])
B = Matrix([
    [0],
    [diff(dv,u)],
    [0],
    [diff(dw,u)]
]).subs([(x,0),(v,0),(q,0),(w,0)])
C = Matrix([[1,0,L,0]])
print("Symbolic A, B and C")
A,B,C

In [ ]:
AA = A.subs(params)
BB = B.subs(params)
CC = C.subs(params)
print("Symbolic A, B and C with paramters substituted in")
AA,BB,CC

In [ ]:
An = np.array(AA).astype(np.float64)
Bn = np.array(BB).astype(np.float64)
Cn = np.array(CC).astype(np.float64)
print("Numeric A, B and C")
An,Bn,Cn

In [ ]:
sys = ss(An, Bn, Cn, [[0]])
Gtemp = tf(sys)
Gtemp

In [ ]:
# Redefine G to get rid of the almost zero coefficient which screws up later calculations
G = tf(chop(Gtemp.num),chop(Gtemp.den))
G

# Root Locus More or Less Fails

In [ ]:
var("s")
((s+2-I)*(s+2+I)*(s+10)).expand()

In [ ]:
Gc = tf([1,1,1],[1])
GcG = series(Gc,G)
r,k = root_locus(GcG,xlim=(-5,5),ylim=(-4,4))

Linear Simulation
---

In [ ]:
K = 0.1
T = feedback(series(tf([K],[1]),GcG))
poles = np.array2string(np.array([N(p,2) for p in pole(T)]))
t,y=step_response(T,np.linspace(0,150,1000))
plt.plot(t,y);
plt.title("Step response of closed loop, linear model\n" + poles)
plt.xlabel("$t$")
plt.xlabel("$y(t)$")

Simulation with nonlinear dynamics
---

In [ ]:
def f_closed_loop(X,t):
    
    x,v,q,w = X

    y = output(x,v,q,w)
    
    # Control law goes here
    K = 0.1
    r = 30
    e = r - y
    u = K*e

    return dynamics(x,v,q,w,u)

tvals = np.linspace(0,100,200)
X = spi.odeint(f_closed_loop,[15,0,0,0],tvals)

show_sim(tvals, X, "Closed Loop, Single Gain");

In [ ]:
anim(X, "Closed Loop, Single Gain")

# Full State Feedback 

Conditioning the new input
---

To apply full state feedback to the model

\begin{align}
\dot\x & = A \x + B u \\
y = C \x
\end{align}

using $u = -K \x + r$ results in the closed loop model

\begin{align}
\dot\x & = (A -BK) \x + B r \\
y = C \x
\end{align}

with $r$ as the new input. By setting the first equation to zero we can find the final value:

\begin{align}
y^* & = - C(A-BK)^{-1}B r \\
    & \triangleq - \kappa r
\end{align}

If we would like $y^*$ to track the input, then we need to scale &mdash; or condition &mdash; $r$ by $- 1/\kappa$. That is, we want 

$$
r = - \frac{1}{\kappa} y^* .
$$

Gain Design
---

Choosing where the poles go depends on performance. A reasonable second order pair of poles plus two more negative poles should work.

In [ ]:
# Pole Placement
K = place(An.tolist(),Bn.tolist(),[-1,-2,-3,-4])
K

In [ ]:
# Condition the input
kappa = np.array(CC*(AA-BB*Matrix(K)).inv()*BB).astype(np.float64)[0,0]
kappa

In [ ]:
def goal(t):
    return 30 if t < 50 else 15
    
def f_fsf(X,t):
    
    x,v,q,w = X
    y = output(x,v,q,w)
    
    # Control Law
    r = - (1/kappa) * goal(t)
    u = - np.dot(K,X)[0] + r

    return dynamics(x,v,q,w,u)

tvals = np.linspace(0,100,200)
X = spi.odeint(f_fsf,[15,0,0,0],tvals)

show_sim(tvals, X, "Full State Feedback");

In [ ]:
anim(X,"Full State Feedback")

# Observer Design

In [ ]:
Lobs = np.transpose(place(np.transpose(An),np.transpose(Cn),[-1+1j,-1-1j,-3,-4]))
Lobs

In the simulation below, we just use an arbitrary input $u = \sin(\omega t)$ to see if the estimated state can track the actual state.

In [ ]:
def f_observer(X,t):
    
    x,v,q,w = X[0:4]
    y = output(x,v,q,w)
    
    xh,vh,qh,wh = X[4:8]
    yh = output(xh,vh,qh,wh)
    
    u = 5 * np.sin(0.2*t)

    dplant = dynamics(x,v,q,w,u)
    dobs = np.matmul(An,[[xh],[vh],[qh],[wh]]) + Bn * u + Lobs * (y-yh)
    
    return np.append(dplant, dobs)

tvals = np.linspace(0,100,200)
X = spi.odeint(f_observer,[10,0,0,0,0,0,0,0],tvals)

show_sim(tvals, X[:,0:4], "State");
show_sim(tvals, X[:,4:8], "Estimate");
show_sim(tvals, X[:,0:4] - X[:,4:8], "Estimate Error");

In [ ]:
anim(X,"Sine Wave Input")

# Full State Feedback with an Observer

In [ ]:
# Place the poles
K = place(An.tolist(),Bn.tolist(),[-0.35,-0.36,-3,-4])
kappa = np.array(CC*(AA-BB*Matrix(K)).inv()*BB).astype(np.float64)[0,0]

# Place the poles for the observer
Lobs = np.transpose(place(np.transpose(An),np.transpose(Cn),[-2+0.01*1j,-2-0.01*1j,-10,-11]))

In [ ]:
# Simulate

def f_fsf_observer(X,t):
    
    x,v,q,w = X[0:4]
    y = output(x,v,q,w)
    
    xh,vh,qh,wh = X[4:8]
    yh = output(xh,vh,qh,wh)
    
    r = - (1/kappa) * goal(t) 
    u = - np.dot(K,[xh,vh,qh,wh])[0] + r
    
    dplant = dynamics(x,v,q,w,u)
    dobs = np.matmul(An,[[xh],[vh],[qh],[wh]]) + Bn * u + Lobs * (y-yh)  
    
    return np.append(dplant, dobs)

tvals = np.linspace(0,100,200)
X = spi.odeint(f_fsf_observer,[15,0,0,0,15,0,0,0],tvals)

show_sim(tvals, X[:,0:4], "FSF + Observer, State and Output");
show_sim(tvals, X[:,4:8], "FSF + Observer, Estimate");
show_sim(tvals, X[:,0:4] - X[:,4:8], "FSF + Observer, Estimate Error");

In [ ]:
anim(X,"FSF + Observer")